# Day 88: Flash Crash Safeguards

Autonomous trading agents, while efficient, can exhibit 'herd behavior'. If a single agent starts liquidating a position rapidly, others might follow its signal, leading to a 'death spiral' or **Flash Crash**. Safety-critical trading systems must have autonomous 'Circuit Breakers'.

In this lab, we implement a **Trading Circuit Breaker** to:
1. **Real-time Drawdown Monitoring**: Calculating price drops within a fixed time window.
2. **Volatility Triggers**: Automatically halting an agent's ability to trade if the market becomes unstable.
3. **Cool-down Periods**: Enforcing a mandatory waiting period for humans to review the market before the AI resumes activity.

In [ ]:
import sys
import os
import time

# Add root directory to sys.path
sys.path.append(os.path.abspath('../../'))

from src.security.circuit_breaker import TradingCircuitBreaker

## 1. Setup the Asset Monitor

We configure a circuit breaker with a 5% threshold over 1 minute.

In [ ]:
cb = TradingCircuitBreaker(max_drawdown_pct=0.05, window_seconds=60, cooldown_seconds=10)
asset = "TESLA-STOCK"

## 2. Simulating a Volatile Market

We feed the AI a series of dropping prices to see when it 'pulls the plug'.

In [ ]:
prices = [200.0, 198.0, 196.0, 194.0, 192.0, 189.0] # Total 5.5% drop

for p in prices:
    if cb.can_trade(asset):
        print(f"Current Price: ${p} -> TRADING ACTIVE")
        cb.update_price(asset, p)
    else:
        print(f"Current Price: ${p} -> SYSTEM HALTED (Safety threshold exceeded)")

## 3. Recovery and Re-Entry

After the `cooldown_seconds` have passed, the agent is allowed to re-enter the market, assuming volatility has stabilized.

In [ ]:
print("Waiting for cooldown...")
time.sleep(11)

print(f"Market Status: {cb.get_status(asset)}")
if cb.can_trade(asset):
    print("Safely resuming trading operations.")

## 🔒 Why this matters

During the 2010 Flash Crash, high-frequency algorithms exacerbated a trillion-dollar market collapse in minutes. Today, safety engineers use more complex versions of these 'liquidation dampeners' to ensure that AI does not create systemic financial risk.